In [1]:
setwd('/import/DB/fel/BJTC-204')

In [2]:
da1 = read.table('./remove1.txt',header =T,sep = '\t')

In [3]:
head(da1,2)

,TYPE,subject,PH,Gender,race,Age,death,GSM
,<chr>,<int>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>
1,MDD,101597,7.05,M,Caucasian,66,0,GSM2431728
2,MDD,101674,6.72,M,Caucasian,65,0,GSM2431731


In [10]:
da2 = read.table('./remove2.txt',header =T,sep = '\t',check.names = F)

In [11]:
head(da2,2)

,ID,102420,101690,105928,103984,106355,107727,102475,107366,107185,...,103620,109436,102608,102579,108022,106082,108291,103917,103211,102349
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10000_at,-0.002368832,0.04337324,0.0523628,0.04786817,0.03918936,0.03359806,0.1133558,0.08256025,-0.06256452,...,0.11859049,0.06649412,-0.3140175,0.1867986,0.03844558,0.087760471,-0.1358281,0.2823591,-0.08261324,-0.04993364
2,10001_at,-0.225477189,0.44336651,0.1176702,0.33329413,0.20939885,0.13793711,0.1152254,0.17699930,0.03033979,...,0.04328463,0.23901771,-0.2168798,0.1154719,0.45487349,0.001328625,-0.2038973,-0.1896431,-0.34764865,0.11837123


In [26]:
dim(da2)

[1] 11979   173

In [13]:
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 4.2.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [16]:
probe = read.table('./probe.txt',header =T,sep = '\t')
head(probe,2)
exp2 = left_join(probe,da2,by = 'ID') %>%  na.omit() 

,ID,SYMBOL
,<chr>,<chr>
1,1_at,A1BG
2,10_at,NAT2


In [17]:

exp2 = exp2[!duplicated(exp2$SYMBOL),]
rownames(exp2) = exp2[,2]
exp3 = exp2[,-c(1,2)] %>% t() %>%   as.data.frame() %>%  tibble::rownames_to_column(.,'GSM')

In [19]:
exp4 = merge(da1,exp3,by.x = 'subject',by.y = 'GSM' )

In [20]:
head(exp4,3)

,subject,TYPE,PH,Gender,race,Age,death,GSM,NAT2,ADA,...,HNRPDL,DMTF1,PPP4R1,CDH1,SLC12A6,ROD1,KCNE2,DGCR2,CASP8AP2,SCO2
,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,101032,MDD,6.51,M,Caucasian,34,1,GSM2431900,0.6483221,-0.08833605,...,-0.11863821,0.23418531,-0.001786439,-0.05258035,0.3886042,0.003912351,0.05898467,-0.2760007,-0.07487967,0.01726185
2,101091,Control,6.64,M,Caucasian,53,0,GSM2431719,0.2656920,0.01237524,...,-0.03964289,-0.05509605,-0.302147249,0.04052146,0.1449086,0.005958422,0.23935568,-0.1894593,-0.22838447,-0.04422506
3,101091,Control,6.64,M,Caucasian,53,0,GSM2431903,0.2656920,0.01237524,...,-0.03964289,-0.05509605,-0.302147249,0.04052146,0.1449086,0.005958422,0.23935568,-0.1894593,-0.22838447,-0.04422506


In [21]:
aaa2 = read.csv('./table1.csv',check.names = F)
aaa2 = aaa2[ complete.cases(aaa2[ , c(7)]),]
head(aaa2,2)

,SUBJECT,DIAGNOSIS,PH,GENDER,RACE,AGE,TOD,death,Toxicology
,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<dbl>,<int>,<chr>
2,54,Control,6.97,M,Caucasian,18,12.23,0,
4,52,Control,6.89,M,Caucasian,55,14.47,0,


In [22]:
colnames(aaa2)[2:6] = c('TYPE','PH','Gender','race','Age')

In [23]:
aaa2$TYPE = gsub('MD','MDD',aaa2$TYPE)

In [27]:
dim(exp4)

[1]   251 11883

In [24]:
exp5 = merge(aaa2, exp4, by = c('TYPE','PH','Gender','race','Age','death')) %>% na.omit()

In [25]:
table(exp5$TYPE)


Control     MDD 
    112      61 

In [29]:
exp5 = exp5[!duplicated(exp5$subject),]

In [30]:
dim(exp5)

[1]    84 11886

In [32]:
table(exp5$TYPE)


Control     MDD 
     51      33 

In [33]:
write.csv(exp5,file = 'TOD.rela.2.csv')

In [34]:
exp5$TOD = ifelse(exp5$TOD < 0,exp5$TOD+24,exp5$TOD)

In [35]:
control = exp5 %>%  dplyr::filter(TYPE == 'Control') %>% dplyr::select(TOD,NAT2:SCO2)  %>%  t() %>%  as.data.frame()
table(exp5$TYPE)
MD = exp5 %>%  dplyr::filter(TYPE != 'Control') %>% dplyr::select(TOD,NAT2:SCO2) %>%  t() %>%  as.data.frame()
head(MD,2)
colnames(control) = control[1,]
control = control[-1,]
colnames(control) = paste0('ZT',colnames(control),'_REP1')
colnames(MD) = MD[1,]
MD = MD[-1,]
colnames(MD) = paste0('ZT',colnames(MD),'_REP1')
colnames(MD)


Control     MDD 
     51      33 

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TOD,11.1500000,1.203e+01,8.6200000,2.1700000,9.5200000,8.5700000,22.9800000,21.9500000,3.3200000,3.5200000,...,14.6800000,9.47000000,5.8700000,15.30000000,2.880000000,4.850000,3.20000000,5.4800000,3.2500000,0.9300000
NAT2,-0.3366612,-6.820e-06,0.2971661,0.2025298,0.6483221,0.3103831,-0.3400011,-0.1747086,-0.3344784,0.2450834,...,-0.2818685,-0.01685043,-0.4305957,0.02316916,0.003915999,-0.199686,-0.05028903,-0.2247213,0.2517364,-0.6745494


[1] "ZT11.15_REP1" "ZT12.03_REP1" "ZT8.62_REP1"  "ZT2.17_REP1"  "ZT9.52_REP1" 
 [6] "ZT8.57_REP1"  "ZT22.98_REP1" "ZT21.95_REP1" "ZT3.32_REP1"  "ZT3.52_REP1" 
[11] "ZT1.17_REP1"  "ZT4.65_REP1"  "ZT19.28_REP1" "ZT22.32_REP1" "ZT6.32_REP1" 
[16] "ZT7.18_REP1"  "ZT4.85_REP1"  "ZT3.55_REP1"  "ZT22.95_REP1" "ZT8.55_REP1" 
[21] "ZT11.17_REP1" "ZT5.58_REP1"  "ZT5.98_REP1"  "ZT14.68_REP1" "ZT9.47_REP1" 
[26] "ZT5.87_REP1"  "ZT15.3_REP1"  "ZT2.88_REP1"  "ZT4.85_REP1"  "ZT3.2_REP1"  
[31] "ZT5.48_REP1"  "ZT3.25_REP1"  "ZT0.93_REP1"

In [36]:
control$GroundTruth = 0
MD$GroundTruth = 0
write.csv(control,file = 'control.csv')
write.csv(MD,file = 'MD.csv')

,ZT11.15_REP1,ZT12.03_REP1,ZT8.62_REP1,ZT2.17_REP1,ZT9.52_REP1,ZT8.57_REP1,ZT22.98_REP1,ZT21.95_REP1,ZT3.32_REP1,ZT3.52_REP1,...,ZT9.47_REP1,ZT5.87_REP1,ZT15.3_REP1,ZT2.88_REP1,ZT4.85_REP1,ZT3.2_REP1,ZT5.48_REP1,ZT3.25_REP1,ZT0.93_REP1,GroundTruth
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NAT2,-0.336661201,-0.000006820,0.29716610,0.202529836,0.648322130,0.31038309,-0.340001146,-0.174708560,-0.33447838,0.245083404,...,-0.016850432,-0.43059571,0.023169164,0.003915999,-0.199685976,-0.05028903,-0.224721266,0.251736429,-0.674549420,0
ADA,0.029589070,-0.020190992,0.16804404,-0.185306192,-0.088336050,0.04560916,0.128605457,-0.114373347,0.16575314,0.101245810,...,-0.135248902,-0.20104390,0.179335902,-0.177955977,-0.155667667,-0.10481710,-0.193399791,-0.090558456,0.095351829,0
CDH2,0.228064131,-0.080117045,0.12859519,0.063235845,-0.161261317,-0.08542227,-0.126233049,-0.260203701,0.05958564,-0.113089010,...,0.039997676,0.01702407,-0.228170399,0.091374945,-0.042860329,0.03610528,0.040233686,-0.041418972,-0.131612329,0
AKT3,-0.029335743,-0.258721684,-0.31401747,0.066494124,0.142178184,-0.25990703,-0.055308427,-0.072836490,0.03063074,0.142318244,...,0.039189363,-0.07960431,0.116198915,0.125029812,0.115472471,0.02671653,0.195896024,0.181149604,0.190400624,0
MED6,-0.076642552,-0.404002792,-0.21687978,0.239017715,0.007261114,0.04825818,0.234895480,0.083285227,0.32796130,0.115924556,...,0.209398849,-0.11958912,-0.016602991,0.026842828,0.237286187,-0.25461862,0.336311069,0.055262079,0.297834612,0
NR2E3,0.014584403,-0.012571960,0.06393712,-0.002646747,-0.031236776,-0.10526517,-0.109650272,-0.090213516,-0.03048112,-0.106993174,...,0.003077232,0.14597541,-0.088359213,0.017930110,-0.017928970,-0.05129297,-0.078981064,-0.037027131,-0.015886592,0
NAALAD2,-0.075170430,-0.175834207,-0.02898025,0.221089624,0.189808918,0.21820471,0.044785969,0.032388275,-0.07668206,0.121139774,...,0.163465470,-0.04960918,-0.063051869,0.013072103,0.094728343,-0.05145997,0.072814324,0.044106011,-0.021173883,0
NAALADL1,-0.020285197,0.005465468,0.09131221,0.200364575,-0.140620882,0.07297548,-0.042047087,0.001293730,0.12347070,-0.085260540,...,-0.022046107,0.13448726,-0.165626289,-0.094299996,-0.011542980,-0.01641036,-0.018615677,-0.052643787,0.030754236,0
CDKN2BAS,0.008844262,-0.000006820,-0.05003856,-0.013816111,0.132632345,0.12641050,-0.187082678,-0.117557034,-0.21805034,-0.143099308,...,0.060924908,0.05504309,-0.107539348,-0.020102724,-0.434195029,0.12426521,-0.010321187,0.168210705,-0.086983975,0
